In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Input ,concatenate
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import concatenate
from tensorflow.keras import regularizers

In [2]:
X_train = load('trainX1.npy')
X_test = load('testX1.npy')
y_train = load('trainY1.npy')
y_test = load('testY1.npy')
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(33104, 128, 6) (33104, 118) (3740, 128, 6) (3740, 118)


In [3]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=42)
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
validation_data=(X_validation, y_validation)

In [4]:
CNN_model= Sequential()
CNN_model.add(Conv1D(filters=32, kernel_size=9, strides=2, activation='relu', input_shape=(n_timesteps,n_features)))
CNN_model.add(MaxPooling1D(pool_size=2 ,strides=2))
CNN_model.add(Dropout(0.5))
CNN_model.add(Conv1D(filters=64, kernel_size=3, strides=1, activation='relu'))
CNN_model.add(Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'))
CNN_model.add(MaxPooling1D(pool_size=2 ,strides=2))
CNN_model.add(Dropout(0.5))
CNN_model.add(Conv1D(filters=128, kernel_size=1, strides=1, activation='relu'))
CNN_model.add(Flatten())
CNN_model.add(Dense(n_outputs, activation='softmax'))
CNN_model.summary()
CNN_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
CNN_model.fit(X_training, y_training, epochs=20, verbose=True, validation_data=(X_validation, y_validation), batch_size=128)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 60, 32)            1760      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 30, 32)            0         
_________________________________________________________________
dropout (Dropout)            (None, 30, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 28, 64)            6208      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 26, 128)           24704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 13, 128)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 128)           0

In [5]:
loss, accuracy = CNN_model.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = CNN_model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9633
Testing Accuracy: 0.8981


In [6]:
CNN_model.trainable = False


In [7]:
LSTM_model= Sequential()
LSTM_model.add(LSTM(512,input_shape=(n_timesteps,n_features),return_sequences=True))
LSTM_model.add(LSTM(256))
LSTM_model.add(Dropout(0.2))
LSTM_model.add(Dense(128, activation='relu'))
LSTM_model.add(Dense(n_outputs, activation='relu'))
LSTM_model.summary()

inputs = Input(shape=(n_timesteps,n_features))
mergedInput= concatenate([CNN_model(inputs),LSTM_model(inputs)])
out = Dense(n_outputs, activation='softmax')(mergedInput)
model3 = Model(inputs,out)
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model3.layers[0].set_weights(LSTM_model.layers[4].get_weights())
model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128, 512)          1062912   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 118)               15222     
Total params: 1,898,486
Trainable params: 1,898,486
Non-trainable params: 0
_________________________________________________________________
Model: "model"
_______________________________________________________________________________________________

In [9]:
history = model3.fit(X_training, y_training, epochs=30, verbose=True, validation_data=(X_validation, y_validation), batch_size=128)

Epoch 1/30
207/207 [==============================] - 610s 3s/step - loss: 0.4464 - accuracy: 0.8826 - val_loss: 0.3403 - val_accuracy: 0.9206
Epoch 2/30
207/207 [==============================] - 501s 2s/step - loss: 0.2858 - accuracy: 0.9249 - val_loss: 0.2436 - val_accuracy: 0.9415
Epoch 3/30
207/207 [==============================] - 501s 2s/step - loss: 0.2088 - accuracy: 0.9446 - val_loss: 0.2356 - val_accuracy: 0.9397
Epoch 4/30
207/207 [==============================] - 505s 2s/step - loss: 0.1783 - accuracy: 0.9517 - val_loss: 0.2075 - val_accuracy: 0.9449
Epoch 5/30
207/207 [==============================] - 499s 2s/step - loss: 0.1408 - accuracy: 0.9615 - val_loss: 0.1571 - val_accuracy: 0.9603
Epoch 6/30
207/207 [==============================] - 502s 2s/step - loss: 0.1237 - accuracy: 0.9650 - val_loss: 0.1329 - val_accuracy: 0.9651
Epoch 7/30
207/207 [==============================] - 500s 2s/step - loss: 0.1092 - accuracy: 0.9694 - val_loss: 0.1257 - val_accuracy: 0.9665

In [10]:
loss, accuracy = model3.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model3.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9778
Testing Accuracy: 0.9152
